# Welcome to Swin Transformer research 🐷👋

В данном notebook'е представлен обзор Swin Transformer (Swin-T), недавно разработанной модели, предназначенной для эффективной обработки изображений в области компьютерного зрения. Notebook охватывает следующие темы:


- Архитектура модели.
- Использование Swin-T.
- Анализ производительности трансформера.
- Возможные способы оптимизации.
  
---
Наиболее широко известной и первой в этом направлении моделью является Vision Transformer (ViT), что придает ей популярность среди исследователей и практиков в области распознавания изображений. Кратко напомним об архитектуре VIT, а в конце выделим достоинства и недостатки Swin-T по сравнению с "трансформером-основателем".

 ## Краткий обзор Vision Transformer (ViT)

Архитектура VIT:
 ![](https://drive.google.com/uc?export=view&id=1lQpVFBRn-bFqd6oGLq2C6VMUADmbJBNY)

В VIT'е исходное изображение разбивается на патчи размером 16x16 пикселей, которые потом преобразуются в векторы и проходят через линейный слой. К этим векторам добавляются обучаемые векторы, которые представляют позиционное встраивание, и отдельный обучаемый вектор, аналогичный CLS-токену в модели BERT.

После следует стандартный Transformer Encoder (N x Multi-Head Attention), и класс изображения предсказывается с помощью небольшого перцептрона, принимающего данные, полученные из CLS-токена. Как и любой трансформер, модель имеет высокие вычислительные требования и требует предварительного обучения на больших наборах данных, таких как JFT-300M.

В области компьютерного зрения существуют задачи, такие как детектирование объектов, где важны детали, а также сегментация, которая требует предсказания на уровне пикселей. Все это требует возможности работы с изображениями высокого разрешения и, следовательно, увеличения размера входных данных. Однако механизм внимания работает с квадратичной сложностью относительно размера входа, что становится проблемой при обработке больших изображений, так как время прямого прохода становится чрезмерно высоким. Кроме того, мелкие детали могут быть потеряны на первом слое, который представляет свертку 16x16 с шагом 16. И эти проблемы способна нивелировать модель Swin-T!

## Swin Transformer архитектура


Рисунок, представленный ниже, демонстрирует архитектуру Swin Transformer, которая является компактной версией Swin-T.

![](https://drive.google.com/uc?export=view&id=1JV_eP-pAVHFUkggSESMs3tGeanAn35Ij)

Структуру трансформера можно разбить на два логических блока:
  1. Подготовительный этап (предобработка изображения и Linear Embedding в Stage 1)
  2. Основное "тело" модели (все остальные Stages с 1-го и до конца)


### Подготовительный этап

В моделях Transformer механизм attention играет существенную роль в анализе последовательностей и установлении связей между элементами данных. Он позволяет модели сконцентрироваться на наиболее значимых областях входных данных для выполнения задач обработки. Несмотря на то, что модель Swin-T предназначена для работы с изображениями вместо текста, основной принцип работы остается неизменным: изображение рассматривается как последовательность слов, что позволяет модели эффективно обрабатывать визуальные данные. Для преобразования изображения в представление "слова" оно сначала разделяется на набор малых патчей с одинаковыми размерами. Каждый патч представляется трехмерным вектором, где первые две компоненты определяют его размерность, а третья компонента представляет цветное пространство RGB. Затем трехмерные векторы патчей преобразуются в одномерные векторы значений, чтобы представить их в качестве "слов" изображения. Далее эти патчи подаются на слой линейного вложения (Linear Embedding), который кодирует в них информацию о позиции в изображении и расширяет их до заданной размерности C.

### Основное "тело" модели


Как можно заметить из рисунка, все stages со второго по четвертый состоят из двух компонент: Patch Merging и Swin Transformer Block. Patch Merging (слияние патчей) является важной операцией. Эта операция позволяет сократить размер модели и уменьшить вычислительную сложность.  Patch Merging выполняет объединение патчей, группируя их в более крупные «смыслывые» блоки, продвигаясь в глубину.  После этого происходит обработка патчей в Swin Transformer Block, структура которого показана на рисунке ниже. Количество трансформерных блоков в каждом этапе (stage) может варьироваться. Обычно начальный этап (stage1) имеет наименьшее количество блоков, а с каждым последующим этапом количество блоков увеличивается. Это связано с постепенным увеличением глубины обрабатываемых данных и необходимостью учесть более сложные зависимости между элементами. Важно отметить, что Swin Block не изменяет размерность входных данных.

![](https://drive.google.com/uc?export=view&id=17wDDbqjAcLrh6qWfohVFxlYRlNgKkXQj)

В конечном итоге, модель Swin-T приводит изображение к иерархическому представлению.


### Подробное объяснение каждого компонента и блока модели


#### Patch Partition
*Patch Partition* - процесс разделения изображения на небольшие непересекающиеся патчи и их преобразование в одномерные векторы. Для модели Swin-T размер одного патча составляет 4х4х3. Тогда для некоторого цветного изображения размером **HxWx3** результатом Patch Partition станет вектор **(H/4)x(W/4)x48** (так как 48=4x4x3).

#### Linear Embedding
*Linear Embedding* - процесс вычисления embedding для результатов Patch Partition. Несмотря на то, что каждый вектор 1x1x48 можно рассматривать как аналог слова для NLP-трансформеров, на слой Linear Embedding, состоящий из одного свёрточного слоя, поступает весь вектор, полученный в результате Patch Partition. Также в этом процессе учитываются positional embedding. В результате исходный вектор расширяется до глубины в 96 единиц, а ширина и высота изменениям не подвергаются.


#### Patch Merging
В сверхточных нейронных сетях, таких как ResNet, дискретизация карт свойств(downsampling of feature maps) выполняется с помощью *операций свертки*.  Операция свертки не всегда является оптимальным выбором для дискретизации. Слишком большой stride приводит к тому, что теряются мелкие детали, а маленький - увеличивает время вычислений.

Поэтому данный тип дискретизации не используется в Swin-T, а применяется *слияние патчей (Patch Merging)*. В этом случае понятие “патч” характеризует наименьший юнит на карте. Иными словами, в карте 14x14 будет 14x14=196 патчей. Чтобы уменьшить выборку карты объектов в n раз, патч merge соединяет объекты каждого набора из n ×n смежных патчей.

В модели Swin-T полагается, что n = 2 и каждая группа содержит по 2x2 соседних патча.
- Шаг 1: Разделения изображение на группы по 2x2.
- Шаг 2: Конкатенация патчей в каждой группе по глубине.
- Шаг 3: Слияние патчей.

Исходный патч размерностью **HxWxC** переходит в **(H/n)x(W/n)x(2*n * C)**.
После выполняется операция нормализации слоя (Layer Normalization) для регуляризации, результат передается в линейный слой (Linear Layer), и размерность становится равной  **(H/n)x(W/n)x(n * C)**, что завершает процесс понижающей дискретизации карты объектов, где размер по сравнению с предыдущей уменьшается в 1/2, а не в 1/2*n.

Зачастую при реализации данного метода применяют следующий алгоритм (см. рисунок ниже):
На Patch Merging слой поступает матрица (тензор) изображения. В ней выделяют следующие компоненты:
- x0 представляет собой под-тензор, полученный из исходного тензора x путем выбора каждого второго элемента начиная с индекса 0 по оси H и каждого второго элемента начиная с индекса 0 по оси W.
- x1 представляет под-тензор, полученный из исходного тензора x путем выбора каждого второго элемента начиная с индекса 1 по оси H и каждого второго элемента начиная с индекса 0 по оси W.
- x2 представляет под-тензор, полученный из исходного тензора x путем выбора каждого второго элемента начиная с индекса 0 по оси H и каждого второго элемента начиная с индекса 1 по оси W.
- x3 представляет под-тензор, полученный из исходного тензора x путем выбора каждого второго элемента начиная с индекса 1 по оси H и каждого второго элемента начиная с индекса 1 по оси W.

Затем полученные под-тензоры объединяются в один тензор. После чего в этом тензоре происходит нормализация значений и снижение размерности.

![](https://drive.google.com/uc?export=view&id=1pxZsD9lNR0f3IB2QnjrsLZAojXvmRidO)

#### LN
*Layer Normalization (LN)* - процесс нормализации полученных данных. В отличие от Batch Normalization, нормализующим значения внутри одного патча, здесь вычисляются среднее значение и стандартное отклонение по каждому слою всех патчей сразу. Разница между BN и LN показана на картинке ниже:

![](https://drive.google.com/uc?export=view&id=1g7leSYKsnl8GhnoYeLpD38nFbOnYPwEq)

#### W-MSA
В стандартной архитектуре трансформера Multi-Head Attention имеет квадратичную сложность, что приводит к негативным эффектам, особенно когда применяется к изображениям с высоким разрешением. Конечно, самым простым и очевидным решением, приходящим на ум, является подсчет Attention для токена не со всеми другими токенами, а только с находящимися в некотором окне фиксированного размера (Window Mutli-Head Attention). Если размерность токенов — C, а размер окна — MxM, то сложности для (Window) Multi-Head Self Attention-ов получаются следующие:

![](https://drive.google.com/uc?export=view&id=1rstTMQRKWW9vRnTk2fyUliti09L3Sm3p)

Tаким образом, при использовании Window Multi-Head Attention вместо квадратичной сложности, время работы Attention становится линейным по размерам входных данных (hw). Однако, такой подход ограничивает общую способность сети представлять данные, поскольку токены из разных окон не взаимодействуют друг с другом.

Для решения этой проблемы, авторы предложили интересный подход. После каждого блока с Window Multi-Head Attention, они добавили дополнительный слой с Attention, где окна были смещены по диагонали. Детальнее будет описано ниже.

#### SW-MSA

SW-MSA расшифровывается как Sliding Window Multi-Head Self-Attention.
SW-MSA разделяет входные данные на последовательные окна фиксированного размера и применяет Multi-Head Self-Attention в каждом окне независимо. Это позволяет модели учитывать локальные зависимости внутри каждого окна и обрабатывать последовательности с переменной длиной. Ползущее окно перемещается циклически по диагонали, чтобы модель могла обработать все соседние связи.

SW-MSA эффективно решает проблему квадратичной сложности, с которой стандартное Multi-Head Self-Attention сталкивается при обработке больших последовательностей. Он также позволяет модели учитывать долгосрочные зависимости внутри каждого окна, что полезно при обработке текстовых данных.

Однако сдвиг окон Attention-а влечет за собой увеличение их числа. Это означает, что если мы используем простое заполнение исходной «карты» признаков нулями, то при вычислении Attention-ов потребуется больше операций, чем если бы мы не применяли сдвиг окон. С целью избежать избыточных вычислений, авторы предложили использовать циклический сдвиг изображения перед вычислением Attention. Это позволяет исключить взаимодействие между несоседними токенами и вычислить маскированный Attention. Такой подход представляет вычислительную эффективность по сравнению с наивным подходом, поскольку количество вычисляемых Attention не увеличивается:

![](https://drive.google.com/uc?export=view&id=1qjcV-hupPiPIxgREMyC9uKAPs9DqlQFW)


В Swin-архитектуре авторы также применили другие типы positional embedding-ов. Они заменили embedding-и на обучаемую матрицу B, известную как relative position bias. Эта матрица добавляется к произведению запроса (query) и ключа (key) перед применением softmax функции. (на практике такое решение выдает более эффективные результаты)

![](https://drive.google.com/uc?export=view&id=1vgE3kCK411AmJTfVXEEootoTISRTCuIz)



#### MLP

MLP (многослойный персептрон) в Swin Transformer используется внутри каждого блока для обработки и преобразования признаков. Он состоит из двух линейных слоев с функцией активации (к примеру GELU), что позволяет модели анализировать нелинейные зависимости между признаками. MLP выполняет две задачи: преобразование признаков внутри каждого блока и объединение информации от разных блоков. Это помогает извлекать высокоуровневые признаки и представления из изображений, улучшая эффективность модели и качество обработки данных.



#### Swin Transformer Block

Swin Transformer Block архитектурно схож с ResNet. Swin-T использует идеи и концепции из ResNet, такие как связи пропуска (skip connections) и многослойные перцептроны (MLP), чтобы обработать и извлечь признаки из входных данных. Однако, Swin-T также добавляет новые элементы, такие как сверточные слои с (Shifted)W-MHA и масштабируемая группирующая свертка (scalable group convolution), чтобы обрабатывать иерархические структуры и улучшить эффективность модели.


***

Стоит отметить, что LN (Layer Normalization) в Swin Transformer Block применяется дважды для нормализации данных. Первая нормализация происходит перед применением операции XOR, а вторая - после. Это делается для того, чтобы стабилизировать обучение и улучшить процесс обратного распространения ошибки. Применение LN перед и после операции XOR позволяет управлять масштабом значений признаков и облегчает градиентный спуск во время обучения модели.


### Сравнение Swin-T с VIT
---
Swin Transformer обладает следющими особенностями:

> 1. Иерархическое представление, помогающее модели улавливать долгосрочные зависимости в изображениях и эффективнее обучаться.
> 2. Более эффективная работа с изображениями высокого разрешения в сравнении с ViT.
> 3. Общая высокая производительность.


Vision Transformer обладает более простой архитектурой, но не обладает основными преимуществами Swin Transformer. Наоборот, ViT очень чувствителен к размеру изображения, и эффективность его работы ухудшается

## Swin Transformer V2

Обновлённая версия модели Swin Transformer, внедрившая некоторые новые идеи, способная тренироваться на изображениях с разрешением вплоть до 1536х1536, с улучшенной стабильностью тренировки и более мягкими требованиями к количеству размеченных данных.

Основные отличия от Swin Transformer V1:

> 1. Слои Layer Norm внутри Swin Transformer Block были поставлены после Attention/MLP слоёв.
> 2. Attention, основанный на матричном умножении, был заменён на взвешенный косинус.
> 3. Изменён подход к формированию relative position bias с parameterized на log-spaced continuous.
> 4. Применение SimMIM.

Первые два нововведения повышают стабильность тренипровки модели. Третье позволяет более эффективно преобразовывать модели, натренированные на изображениях с низким разрешением, для задач, требующих на вход изображения более высокого разрешения. Четвёртое уменьшает необходимость в большом количестве размеченных данных.

![](https://drive.google.com/uc?export=view&id=1ERg-tGuvmxHlD5ZgESedCz8A9iJKkTU_)
